# Instructions
Set up an array of ten random numbers between 100 and 500 (just one array with 10 numbers).
Calculate the factorial of each of the numbers using a simple FOR loop.  Please capture the start/end times and store the results in an array, or linked list, or dictionary.  
Calculate the factorial of each of the numbers using recursion.  Please capture the start/end times and store the results in an array, or linked list, or dictionary.
Organize the results of the study into a table showing the number, it's factorial, the time to calculate the factorial using recursion, the time to calculate factorial using FOR loop, and the difference between times.
Use python matplotlib or seaborn to generate a graph that has number on the horizontal axis and with execution time in milliseconds on the vertical axis. The plot should show execution times of recursion and the loop.

In [5]:
import numpy as np
import pandas as pd
import time
import string
import random
import matplotlib.pyplot as plt


plt.style.use('bmh')
np.random.seed = 34 #set seed
LOG = {} # Define Log

arr_size = 10
array_test = np.random.randint(100,500,arr_size)

### Timing Mechanism
The timing mechanism used in this experiment wraps functions and times their overall time to return function output.
A dictionary "LOG" is updated with the output times of the functions

In [6]:
def timeit(log):
    """ Wraps functions needing timing provided a dictionary to dump into based on function name"""
    def log_it(func):
        def wrapped(*args, **kw):
            ts = time.time_ns() // 1000000 
            result = func(*args, **kw)
            te = time.time_ns() // 1000000 
            if func.__name__ in log.keys():
                log[func.__name__].append(te-ts)
            else:
                log[func.__name__] = [te-ts]
            return result
        return wrapped
    return log_it

In [5]:
@timeit(LOG)
def factorial(number):
    value = 1
    for e in range(1,number+1):
        value *= e
    return value


In [6]:
facs = []
for test_element in array_test:
    f = factorial(test_element)
    facs.append(f)

In [7]:
LOG

{'factorial': [7, 0, 0, 0, 0, 0, 0, 1, 0, 0]}

In [7]:
@timeit(LOG)
def factorial_full_recursive_stack(n):
    def factorial_recur(n):
        if n == 1:
            return n
        
        return n*factorial_recur(n-1)  

    return factorial_recur(n)

In [8]:
facs_recur = []
for test_element in array_test:
    f = factorial_full_recursive_stack(test_element)
    facs_recur.append(f)

<ipython-input-7-459ad35cd5cf>:7: RuntimeWarning: overflow encountered in long_scalars
  return n*factorial_recur(n-1)


In [9]:
LOG


{'factorial_full_recursive_stack': [2, 1, 2, 0, 1, 0, 0, 0, 0, 0]}